&nbsp;

# 14 bis - Extraction d'un état réduit par sélection
---

&nbsp;

## 1. Introduction et justification

> #### Pourquoi extraire un état réduit de cette manière ?

Quand on construit notre état réduit à partir de la PCA/EOF, on construit des combinaisons linéaires des variables originales. Les modes (EOF1, EOF2, etc.) n'existent pas physiquement et permettent surtout de maximiser la variance expliquée.

Le défaut, c'est que dans un projet où notre objectif est de capturer la dynamique thermique d'une zone géographique définie, entraîner notre modèle sur des données synthétiques peut devenir un obstacle à l'interprétabiltié causale et physique.

&nbsp;

Dans l'optique de pouvoir comparer avec notre état réduit initial, nous allons réaliser une sélection de *features* (une seconde famille de méthode pour réduire un jeu de donnée). Ces méthodes permettent de conserver les variables à variance élevée (celle qui sont les plus explicatives) ou à variance conditionnelle.

&nbsp;

---

&nbsp;

## 2. Théorie

> #### Qu'est-ce que la régularisation et pourquoi est-elle nécessaire ?

Dans les systèmes physiques à haute dimension, on observe souvent que le nombre de variables $N$ est grand, le nombre d'observations $T$ est limité et que les variables sont fortement corrélées. Ce contexte pose un problème de régression classique ($y = X\beta + \varepsilon$) qui mène le plus souvent à des approches erronées (solutions instables, sur-apprentissage et/ou faible capacité à généraliser). Lesquelles, dans notre cas, sont :

- un cas critique avec $N \geq T$, car la solution $\hat{\beta}$ n'existe pas ou instable
- une multicolinéarité structurelle majeure et problématique, car le modèle n'arrive pas à "choisir" entre plusieurs variables redondantes (points spatiaux voisins fortement corrélés)

La régularisation est une méthode solutionnant cette problématique. Celle-ci consiste à rajouter une contrainte supplémentaire à la régression classique dans le but de stabiliser la solution, comme suit :

$$
\underset{\beta}{\text{min}} \space || y - X\beta||²_2 + \lambda \mathcal{R}(\beta)
$$

Il existe 3 principaux types de régressions classiques :

- Régularisation L2 dite *Ridge* :

$$
\mathcal{R}(\beta) = ||\beta||²_2
$$

$\to$ réduit l'amplitude des coefficients et stabilise la solution (utile pour la prédiction).

- Régularisation L1 dite *LASSO* :

$$
\mathcal{R}(\beta) = ||\beta||_1 = \underset{j}{\sum}\beta_j
$$
$\to$ pousse certains coefficients à zéro et induit donc une sélection de variables (transformant le problème de régression en problème de sélection, notre cas).

- Régularisation L1 + L2 dite *Elastic Net* :

$$
\mathcal{R}(\beta) = \alpha ||\beta||_1 + (1-\alpha)||\beta||²_2
$$
$\to$ compromission entre stabilité et parcimonie.

&nbsp;

De nombreux problèmes physiques, la fonction capturant la dynamique dépend généralement d'un petit nombre de variables où les interactions sont parcimonieuses. La régularisation L1 nous permet donc d'identifier les variables qui gouvernent tout en conservant une interprétabilité physique claire.

Notre motivation derrière l'utilisation de cette combinaison d'approches est d'utiliser l'EOF pour "comprendre" la structure globale, mais en définissant notre état réduit uniquement dans l'espace physique.

&nbsp;

---

&nbsp;

## 3. Implémentation d'une méthode de sélection

&nbsp;

Nous allons implémenter une combinaison simple (SINDy + Lasso) pour réduire notre jeu de données d'anomalies désaisonnalisées par la sélection.

On commence par charger notre jeu de données post analyse statistique et importons les librairies qui nous serons utiles. On *reshape* le champ de SST en une matrice 2D en "empilant" les dimensions spatiales (ainsi chaque localisation correspond à un élément de la colonne spatiale).

Le champ SST est initialement un champ spatio-temporel : 

$$
\text{SST}(t,\phi,\lambda)
$$

Et comme dans notre précédente extraction, nous le reformulons en une matrice de données :

$$
X \in \mathbb{R}^{T\times N}
$$

où :
- T = nombre d'instants temporels
- N = nombre de point spatiaux

Chaque colonne correspondra ainsi à une variable physique réelle : 

$$
X(t) = [x_1(t), x_2(t), \dots, x_N(t)]
$$

&nbsp;

In [4]:
import xarray as xr
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso

ds = xr.open_dataset("data/processed/sstDeseasonalizedCOPERNICUS20102019.nc")

sst = ds["analysed_sst"]

# Shaping from (time, lat, lon) to (time, space)
sstStacked = sst.stack(space=("latitude", "longitude"))
sstStacked = sstStacked.dropna("space")

X = sstStacked.values  # (T, Nspace)
dt = 1.0  # time step in days

&nbsp;

En suite, on applique une standardisation colonne par colonne. C'est indispensable car :

- PCA/EOF maximise la variance :
$$
\text{max Var}(u^{\perp}X)
$$
$\to$ sans normalisation, les régions à forte variance dominent artificiellement.
- Lasso résout (voir la partie théorie de la méthode utilisée):
$$
\underset{\beta}{\text{min}}||y - X\beta||²_2 + \alpha||\beta||_1
$$
$\to$ cela correspond à une régression linéaire classique, plus un facteur dit "pénalisant" $\mathcal{l}_1$ qui dépend directement de l'échelle.

Par une normalisation du type :

$$
\tilde{x}(t) = \frac{x(t) - \mu}{\sigma}
$$

$\to$ on garantie que la sélection repose sur la dynamique plutôt que sur l'amplitude brute.

&nbsp;

In [5]:
# We use the StandardScaler wich simply removes the mean and scales to unit variance
scaler = StandardScaler()
XScaled = scaler.fit_transform(X) # fit_transform applies the scaling choosen

&nbsp;

> #### Pourquoi réutilise-t-on la PCA ?

C'est le *twist*. On applique la PCA non pas pour utiliser les modes résultant comme variables d'état de notre état réduit mais plutôt comme outil intermédiaire. Elles nous servent uniquement commme élément d'observation/comparaison pour, par la suite, identifier les variables physiques du jeu qui structurent la dynamique globale (puis à les sélectionner). Pour plus de précisions sur la méthode de la PCA, voir les notebooks 10 et 14.

&nbsp;

In [6]:
# We perform PCA like notebook 14pcaReduction.ipynb

nComponents = 150 # We choose K = 100 principal components as before    
pca = PCA(n_components=nComponents, svd_solver="full") # We choose the full SVD solver as we did before

PCs = pca.fit_transform(XScaled)       # (T, K)
EOFs = pca.components_.T               # (Nspace, K)
explainedVar = pca.explained_variance_ratio_

&nbsp;

Dans cette étape, nous classons les variables physiques en fonction de leur contribution aux modes de variabilité dominants. L'idée est de mesurer la contribution de la variable $x_j$ au mode $k$, où chaque EOF suit :

$$
u_k = (u_{1k}, u_{2k}, \dots, u_{Nk})
$$

Ainsi, on définit un score comme : 

$$
\mathcal{S}_j = \overset{K}{\underset{k=1}{\sum}}\lambda_k |u_{jk}|
$$

où:
- $\lambda_k$ = variance expliquée du mode $k$
- $|u_{jk}|$ = coefficients de $u_k$

Une variable sera décelée important si elle apparaît fortement et/ou dans les modes (énergétiquement) dominants.

&nbsp;

In [7]:
topKModes = 30
importance = np.zeros(EOFs.shape[0])

for k in range(topKModes):
    importance += explainedVar[k] * np.abs(EOFs[:, k])

nCandidates = 20
candidateIndices = np.argsort(importance)[-nCandidates:]

&nbsp;

On cherche un signal directionnel pour la sélection dynamique. Pour nourrir ce but, on approxime par dérivation par différences finies centrées. De cette manière, on fait l'hypothèse implicite que le système suit une dynamique continue de nature :

$$
\frac{dX}{dt} = F(X)
$$

&nbsp;

In [8]:
dXdt = np.gradient(XScaled, dt, axis=0)


&nbsp;

On choisit le premier PC comme cible dynamique qui sera notre critère principal de sélection. C'est lui qui capture la plus grande part de variance et donc la dynamique dominante du système. Il vérifie :

$$
a_1(t) = u_1^\perp X(t)
$$

Puis on résoud le problème mathématique, au coeur de la méthode LASSO, suivant :

$$
\underset{\beta}{\text{min}} \left \| \frac{da_1}{dt} - X_c\beta \right \| ²_2 + \alpha||\beta||_1 
$$

Pour plus d'explication de la méthode LASSO, voir la partie théorie.

&nbsp;

In [9]:
# We choose the first principal component as the target variable
target = PCs[:, 0]
dTargetdt = np.gradient(target, dt)

XCandidates = XScaled[:, candidateIndices]

# We resolve dTargetdt as a sparse linear combination of the candidate predictors
# In other words, we solve the equation dTargetdt = Lasso(XCandidates)
lasso = Lasso(alpha=0.005, max_iter=10000)
lasso.fit(XCandidates, dTargetdt)

# We select the predictors with non-zero coefficients because they are the selected variables
selectedMask = np.abs(lasso.coef_) > 1e-6
selectedIndices = candidateIndices[selectedMask]

&nbsp;

Résultat, nous avons :

$$
\frac{da_1}{dt} \approx \underset{j \in S}{\sum} \beta_j x_j
$$

On peut désormais construire notre second état réduit à partir des indices sélectionnés *a posteriori*. L'état réduit est :

$$
X_r(t) = [x_{j_1}(t), x_{j_2}(t), \dots, x_{j_m}(t)]
$$

&nbsp;

In [10]:
XReducedScaled = XScaled[:, selectedIndices]
dXdtReduced = np.gradient(XReducedScaled, dt, axis=0)

**Propriété de notre état**

- aucune combinaison linéaire
- aucune projection
- variables physiques uniquement

Nous n'avons plus qu'à sauvegarder l'output de notre programme en fichier NetCDF.

In [11]:
# We build a new xarray Dataset to store the reduced state

dsReduced = xr.Dataset(
    {
        "XReduced": (("time", "space"), XReducedScaled),
        "dXdtReduced": (("time", "space"), dXdtReduced),
    },
    coords={
        "time": ds["time"],
        "space": selectedIndices,
    },
    attrs={
        "description": "Reduced physical state obtained via EOF-based ranking and sparse dynamic selection"
    }
)

dsReduced.to_netcdf("data/processed/sstReducedState2COPERNICUS20102019.nc") # then save thanks to xarray to_netcdf function


&nbsp;

**Structure du fichier *sstReducedState2COPERNICUS20102019.nc***

- XReduced → observables physiques originales et dynamiquement pertinent sélectionnés

- dXdtReduced → dérivée temporelle discrète des variables d'état réduit calculée par différences centrées finies

- coords :

    - time : axe temporel original

    - space : indices des points spatiaux du couple $\text{(latitude, longitude)}$